In [4]:
import pandas as pd
import glob, os, re

folder_path = r"C:\Users\black\Documents\SINCA files"      # change if needed
out_path = r"C:\Users\black\Documents\SINCA2\Data_Pollution_cleaned.csv"

all_files = glob.glob(os.path.join(folder_path, "*.csv"))
dfs = []

def merge_split_decimals(df: pd.DataFrame) -> pd.DataFrame:
    """
    Merge adjacent 'Unnamed:*' column into the left neighbor when left looks like integer
    and the unnamed right looks like a fractional part (digits only).
    """
    cols = list(df.columns)
    to_drop = []
    # Work as strings
    for c in cols:
        df[c] = df[c].astype(str)
    df = df.replace({"": pd.NA, "nan": pd.NA, "None": pd.NA})
    for i, c in enumerate(cols):
        if re.match(r"^Unnamed: ?\d+$", str(c)) and i > 0:
            left = cols[i-1]
            left_is_int = df[left].str.fullmatch(r"-?\d+").fillna(False)
            right_is_frac = df[c].str.fullmatch(r"\d+").fillna(False)
            mask = left_is_int & right_is_frac
            if mask.any():
                # preserve leading zeros on fractional part
                df.loc[mask, left] = df.loc[mask, left].astype(str) + "." + df.loc[mask, c].astype(str)
                to_drop.append(c)
    if to_drop:
        df = df.drop(columns=list(dict.fromkeys(to_drop)), errors="ignore")
    return df

def fix_decimal_cell(val):
    """Turn '13,4399' -> '13.4399', '69 9047' -> '69.9047', leave others unchanged."""
    if pd.isna(val):
        return val
    s = str(val).strip()
    # comma decimal
    if re.fullmatch(r"-?\d+,\d+", s):
        return s.replace(",", ".")
    # space decimal like '69 9047' (one or more spaces)
    if re.fullmatch(r"-?\d+\s+\d+", s):
        return re.sub(r"\s+", ".", s)
    return s

for file in all_files:
    fname = os.path.basename(file).replace(".csv", "")
    # split filename into medida and centro (keeps everything after first '__' as centro)
    medida, centro = fname.split("__", 1)

    # Read as strings to avoid early coercion
    df = pd.read_csv(file, sep=";", dtype=str, engine="python")

    # If CSV was ragged and created Unnamed columns, try to merge split decimals
    df = merge_split_decimals(df)

    # Trim whitespace and normalize empties
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df = df.replace({"": pd.NA})

    # Fix decimal patterns in every column where it's needed
    for col in df.columns:
        # quick check: only act if there is evidence of comma-decimal or spaced-decimal
        col_sample = df[col].dropna().astype(str)
        if col_sample.empty:
            continue
        if col_sample.str.contains(r",").any() or col_sample.str.contains(r"\d+\s+\d+").any():
            df[col] = df[col].apply(fix_decimal_cell)
            # try to convert to numeric (if appropriate)
            df[col] = pd.to_numeric(df[col], errors="ignore")

    # Attach metadata
    df["Medida"] = medida
    df["Centro"] = centro.replace("_", " ")

    dfs.append(df)

# Concatenate all files
combined_df = pd.concat(dfs, ignore_index=True)

# Convert FECHA (YYMMDD) to datetime (Excel will read it fine)
if "FECHA (YYMMDD)" in combined_df.columns:
    combined_df["FECHA (YYMMDD)"] = pd.to_datetime(
        combined_df["FECHA (YYMMDD)"].astype(str).str.strip(), format="%y%m%d", errors="coerce"
    )

# Drop unwanted columns
combined_df = combined_df.drop(columns=["HORA (HHMM)", "Unnamed: 5"], errors="ignore")

# Normalize and convert the registro columns
for col in ["Registros validados", "Registros preliminares", "Registros no validados"]:
    if col in combined_df.columns:
        combined_df[col] = (
            combined_df[col].astype(str)
            .str.replace(",", ".", regex=False)
            .str.replace(r"\s+", ".", regex=True)
        )
        combined_df[col] = pd.to_numeric(combined_df[col], errors="coerce")

# --- NEW: create total registros ---
registro_cols = ["Registros validados", "Registros preliminares", "Registros no validados"]
available_cols = [c for c in registro_cols if c in combined_df.columns]

if available_cols:
    combined_df["Total registros"] = combined_df[available_cols].sum(axis=1, skipna=True)

    # Drop Medida+Centro groups where all Total_registros are NaN or 0
    combined_df = combined_df.groupby(["Medida", "Centro"], group_keys=False).filter(
        lambda g: not g["Registros validados"].fillna(0).eq(0).all()
    )

# Reset index for a clean output
combined_df = combined_df.reset_index(drop=True)

# --- NEW: aggregate to daily min, max, mean for each Medida–Centro ---
# First identify the column that contains the measure values.
# I’ll assume it’s "Registros validados" (adjust if needed).
measure_col = "Registros validados"

if measure_col in combined_df.columns:
    daily_df = (
        combined_df
        .groupby(["FECHA (YYMMDD)", "Medida", "Centro"], as_index=False)
        .agg(
            Min_val=(measure_col, "min"),
            Max_val=(measure_col, "max"),
            Mean_val=(measure_col, "mean")
        )
    )
else:
    daily_df = combined_df.copy()


# Save cleaned output
daily_df.to_csv(out_path, index=False)
print("Saved cleaned file to:", out_path)




C:\Users\black\AppData\Local\Temp\ipykernel_28484\2716048680.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  left_is_int = df[left].str.fullmatch(r"-?\d+").fillna(False)
C:\Users\black\AppData\Local\Temp\ipykernel_28484\2716048680.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  right_is_frac = df[c].str.fullmatch(r"\d+").fillna(False)
C:\Users\black\AppData\Local\Temp\ipykernel_28484\2716048680.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) el

Saved cleaned file to: C:\Users\black\Documents\SINCA2\Data_Pollution_cleaned.csv


In [3]:
if available_cols:
    combined_df["Registros validados"] = combined_df[available_cols].sum(axis=1, skipna=True)

    # Drop Medida+Centro groups where all Total_registros are NaN or 0
    combined_df = combined_df.groupby(["Medida", "Centro"], group_keys=False).filter(
        lambda g: not g["Registros validados"].fillna(0).eq(0).all()
    )

# Reset index for a clean output
combined_df = combined_df.reset_index(drop=True)

# --- NEW: aggregate to daily min, max, mean for each Medida–Centro ---
# First identify the column that contains the measure values.
# I’ll assume it’s "Registros validados" (adjust if needed).
measure_col = "Registros validados"

if measure_col in combined_df.columns:
    daily_df = (
        combined_df
        .groupby(["FECHA (YYMMDD)", "Medida", "Centro"], as_index=False)
        .agg(
            Min_val=(measure_col, "min"),
            Max_val=(measure_col, "max"),
            Mean_val=(measure_col, "mean")
        )
    )
else:
    daily_df = combined_df.copy()


# Save cleaned output
daily_df.to_csv(out_path, index=False)

In [ ]:
import pandas as pd
import glob, os, re

folder_path = r"C:\Users\black\Documents\SINCA files"      # change if needed
out_path = r"C:\Users\black\Documents\SINCA2\Data_Pollution_cleaned2.csv"

all_files = glob.glob(os.path.join(folder_path, "*.csv"))
dfs = []

def merge_split_decimals(df: pd.DataFrame) -> pd.DataFrame:
    """
    Merge adjacent 'Unnamed:*' column into the left neighbor when left looks like integer
    and the unnamed right looks like a fractional part (digits only).
    """
    cols = list(df.columns)
    to_drop = []
    # Work as strings
    for c in cols:
        df[c] = df[c].astype(str)
    df = df.replace({"": pd.NA, "nan": pd.NA, "None": pd.NA})
    for i, c in enumerate(cols):
        if re.match(r"^Unnamed: ?\d+$", str(c)) and i > 0:
            left = cols[i-1]
            left_is_int = df[left].str.fullmatch(r"-?\d+").fillna(False)
            right_is_frac = df[c].str.fullmatch(r"\d+").fillna(False)
            mask = left_is_int & right_is_frac
            if mask.any():
                # preserve leading zeros on fractional part
                df.loc[mask, left] = df.loc[mask, left].astype(str) + "." + df.loc[mask, c].astype(str)
                to_drop.append(c)
    if to_drop:
        df = df.drop(columns=list(dict.fromkeys(to_drop)), errors="ignore")
    return df

def fix_decimal_cell(val):
    """Turn '13,4399' -> '13.4399', '69 9047' -> '69.9047', leave others unchanged."""
    if pd.isna(val):
        return val
    s = str(val).strip()
    # comma decimal
    if re.fullmatch(r"-?\d+,\d+", s):
        return s.replace(",", ".")
    # space decimal like '69 9047' (one or more spaces)
    if re.fullmatch(r"-?\d+\s+\d+", s):
        return re.sub(r"\s+", ".", s)
    return s

for file in all_files:
    fname = os.path.basename(file).replace(".csv", "")
    # split filename into medida and centro (keeps everything after first '__' as centro)
    medida, centro = fname.split("__", 1)

    # Read as strings to avoid early coercion
    df = pd.read_csv(file, sep=";", dtype=str, engine="python")

    # If CSV was ragged and created Unnamed columns, try to merge split decimals
    df = merge_split_decimals(df)

    # Trim whitespace and normalize empties
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df = df.replace({"": pd.NA})

    # Fix decimal patterns in every column where it's needed
    for col in df.columns:
        # quick check: only act if there is evidence of comma-decimal or spaced-decimal
        col_sample = df[col].dropna().astype(str)
        if col_sample.empty:
            continue
        if col_sample.str.contains(r",").any() or col_sample.str.contains(r"\d+\s+\d+").any():
            df[col] = df[col].apply(fix_decimal_cell)
            # try to convert to numeric (if appropriate)
            df[col] = pd.to_numeric(df[col], errors="ignore")

    # Attach metadata
    df["Medida"] = medida
    df["Centro"] = centro.replace("_", " ")

    dfs.append(df)

# Concatenate all files
combined_df = pd.concat(dfs, ignore_index=True)

# Convert FECHA (YYMMDD) to datetime (Excel will read it fine)
if "FECHA (YYMMDD)" in combined_df.columns:
    combined_df["FECHA (YYMMDD)"] = pd.to_datetime(
        combined_df["FECHA (YYMMDD)"].astype(str).str.strip(), format="%y%m%d", errors="coerce"
    )

# Drop unwanted columns
combined_df = combined_df.drop(columns=["HORA (HHMM)", "Unnamed: 5"], errors="ignore")

# Normalize and convert the registro columns
for col in ["Registros validados", "Registros preliminares", "Registros no validados"]:
    if col in combined_df.columns:
        combined_df[col] = (
            combined_df[col].astype(str)
            .str.replace(",", ".", regex=False)
            .str.replace(r"\s+", ".", regex=True)
        )
        combined_df[col] = pd.to_numeric(combined_df[col], errors="coerce")

# --- NEW: create total registros ---
registro_cols = ["Registros validados", "Registros preliminares", "Registros no validados"]
available_cols = [c for c in registro_cols if c in combined_df.columns]

if available_cols:
    combined_df["Total_registros"] = combined_df[available_cols].sum(axis=1, skipna=True)

    # Drop Medida+Centro groups where all Total_registros are NaN or 0
    combined_df = combined_df.groupby(["Medida", "Centro"], group_keys=False).filter(
        lambda g: not g["Total_registros"].fillna(0).eq(0).all()
    )

# Reset index for a clean output
combined_df = combined_df.reset_index(drop=True)
# --- NEW: classify registro type per day per Centro-Medida ----
import numpy as np

# Ensure all registro columns exist
registro_cols = ["Registros validados", "Registros preliminares", "Registros no validados"]
for col in registro_cols:
    if col not in combined_df.columns:
        combined_df[col] = np.nan

# Create a categorical variable with priority:
# validados > preliminares > no validados > sin registro
def classify_registro(row):
    if pd.notna(row["Registros validados"]) and row["Registros validados"] > 0:
        return "Validados"
    elif pd.notna(row["Registros preliminares"]) and row["Registros preliminares"] > 0:
        return "Preliminares"
    elif pd.notna(row["Registros no validados"]) and row["Registros no validados"] > 0:
        return "No validados"
    else:
        return "Sin registro"

combined_df["Tipo_registro"] = combined_df.apply(classify_registro, axis=1)

# Aggregate to *one registro type per day* for each Centro-Medida
daily_type = (
    combined_df
    .groupby(["Medida", "Centro", "FECHA (YYMMDD)"], as_index=False)["Tipo_registro"]
    .first()          # after cleaning, there should be one row per day, but just in case
)

# --- Create full date range (2557 days) ---
full_range = pd.date_range("2018-01-01", "2024-12-31", freq="D")
# --- Create full date range (2557 days) ---

# Build a frame with all Centro–Medida–date combinations
full_index = (
    combined_df[["Medida", "Centro"]]
    .drop_duplicates()
    .assign(key=1)
    .merge(
        pd.DataFrame({"FECHA (YYMMDD)": full_range, "key": 1}),
        on="key"
    )
    .drop(columns="key")
)

# Merge with daily registro classification
full_df = full_index.merge(
    daily_type,
    how="left",
    on=["Medida", "Centro", "FECHA (YYMMDD)"]
)

# Fill missing registro type
full_df["Tipo_registro"] = full_df["Tipo_registro"].fillna("Sin registro")

# --- Count & percentage for each Centro–Medida ---
summary = (
    full_df
    .groupby(["Medida", "Centro", "Tipo_registro"])
    .size()
    .reset_index(name="N_dias")
)

total_days = len(full_range)  # = 2557
summary["Porcentaje"] = (summary["N_dias"] / total_days) * 100

# pivot table to export
summary_pivot = summary.pivot_table(
    index=["Medida", "Centro"],
    columns="Tipo_registro",
    values=["N_dias", "Porcentaje"],
    fill_value=0
).reset_index()

# --- Export CSV ---
out_summary = r"C:\Users\black\Documents\SINCA2\Resumen_TipoRegistro_por_CentroMedida.csv"
summary_pivot.to_csv(out_summary, index=False)

print("✅ Exported final summary to:", out_summary)


C:\Users\black\AppData\Local\Temp\ipykernel_3788\226124916.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  left_is_int = df[left].str.fullmatch(r"-?\d+").fillna(False)
C:\Users\black\AppData\Local\Temp\ipykernel_3788\226124916.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  right_is_frac = df[c].str.fullmatch(r"\d+").fillna(False)
C:\Users\black\AppData\Local\Temp\ipykernel_3788\226124916.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


ValueError: cannot insert Centro, already exists

In [2]:

# Build a frame with all Centro–Medida–date combinations
full_index = (
    combined_df[["Medida", "Centro"]]
    .drop_duplicates()
    .assign(key=1)
    .merge(
        pd.DataFrame({"FECHA (YYMMDD)": full_range, "key": 1}),
        on="key"
    )
    .drop(columns="key")
)

# Merge with daily registro classification
full_df = full_index.merge(
    daily_type,
    how="left",
    on=["Medida", "Centro", "FECHA (YYMMDD)"]
)

# Fill missing registro type
full_df["Tipo_registro"] = full_df["Tipo_registro"].fillna("Sin registro")

# --- Count & percentage for each Centro–Medida ---
summary = (
    full_df
    .groupby(["Medida", "Centro", "Tipo_registro"])
    .size()
    .reset_index(name="N_dias")
)

total_days = len(full_range)  # = 2557
summary["Porcentaje"] = (summary["N_dias"] / total_days) * 100

# pivot table to export
summary_pivot = summary.pivot_table(
    index=["Medida", "Centro"],
    columns="Tipo_registro",
    values=["N_dias", "Porcentaje"],
    fill_value=0
).reset_index()

# --- Export CSV ---
out_summary = r"C:\Users\black\Documents\SINCA2\Resumen_TipoRegistro_por_CentroMedida.csv"
summary_pivot.to_csv(out_summary, index=False)

print("✅ Exported final summary to:", out_summary)

✅ Exported final summary to: C:\Users\black\Documents\SINCA2\Resumen_TipoRegistro_por_CentroMedida.csv


In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.ops import nearest_points
import numpy as np


def build_df(entidades_path, centros_path, csv_path, radius_km=2):
    # Load shapefiles
    entidades = gpd.read_file(entidades_path)
    centros = gpd.read_file(centros_path)

    # Load allowed Estaciones from CSV
    valid_estaciones = pd.read_csv(csv_path)["Estación"].astype(str).unique()

    # Keep only matching centros
    centros = centros[centros["Estación"].astype(str).isin(valid_estaciones)].copy()

    # Project to metric CRS
    entidades = entidades.to_crs(epsg=32719)  # Adjust CRS if needed
    centros = centros.to_crs(entidades.crs)

    records = []

    for idx, ent in entidades.iterrows():
        ent_geom = ent.geometry

        # Compute distances
        centros["dist_m"] = centros.geometry.distance(ent_geom)
        nearby = centros[centros["dist_m"] <= radius_km * 1000].copy()

        if nearby.empty:
            records.append({
                "Entidad": ent.get("name", idx),  # adjust col name
                "Estación": ".",
                "cod_comuna": ent["CUT"],
                "Pers": ent["TOTAL_PERS"],
                "weight": 0
            })
        else:
            # Inverse distance weights
            nearby["inv_dist"] = 1 / nearby["dist_m"]
            nearby["weight"] = nearby["inv_dist"] / nearby["inv_dist"].sum()

            for _, cen in nearby.iterrows():
                records.append({
                    "Entidad": ent.get("name", idx),
                    "Estación": cen["Estación"],
                    "cod_comuna": ent["CUT"],
                    "Pers": ent["TOTAL_PERS"],  # keep original name
                    "weight": cen["weight"]
                })

    return pd.DataFrame(records)

def build_df2(entidades_path, centros_path, csv_path, radius_km=2):
    # Load shapefiles
    entidades = gpd.read_file(entidades_path)
    centros = gpd.read_file(centros_path)

    # Load allowed Estaciones from CSV
    valid_estaciones = pd.read_csv(csv_path)["Estación"].astype(str).unique()

    # Keep only matching centros
    centros = centros[centros["Estación"].astype(str).isin(valid_estaciones)].copy()

    # Project to metric CRS
    entidades = entidades.to_crs(epsg=32719)  # Adjust CRS if needed
    centros = centros.to_crs(entidades.crs)

    records = []

    for idx, ent in entidades.iterrows():
        ent_geom = ent.geometry

        # Compute distances
        centros["dist_m"] = centros.geometry.distance(ent_geom)
        nearby = centros[centros["dist_m"] <= radius_km * 1000].copy()

        if nearby.empty:
            records.append({
                "Entidad": ent.get("name", idx),  # adjust col name
                "Estación": ".",
                "cod_comuna": ent["COD_COMUNA"],
                "Pers": ent["TOTAL_PERS"],
                "weight": 0
            })
        else:
            # Inverse distance weights
            nearby["inv_dist"] = 1 / nearby["dist_m"]
            nearby["weight"] = nearby["inv_dist"] / nearby["inv_dist"].sum()

            for _, cen in nearby.iterrows():
                records.append({
                    "Entidad": ent.get("name", idx),
                    "Estación": cen["Estación"],
                    "cod_comuna": ent["COD_COMUNA"],
                    "Pers": ent["TOTAL_PERS"],  # keep original name
                    "weight": cen["weight"]
                })

    return pd.DataFrame(records)


# ---- USAGE ----
df21 = build_df(r"E:\Sick leave\microdatos_manzana\Centroide\Centroides_Manzana_32718.shp", r"E:\Sick leave\microdatos_manzana\Centroide\Coordenadas_LatLon_32718.shp", r"C:\Users\black\Documents\SINCA2\centros2.csv", radius_km=10)
df22 = build_df(r"E:\Sick leave\microdatos_manzana\Centroide\Centroides_Manzana_32719.shp", r"E:\Sick leave\microdatos_manzana\Centroide\Coordenadas_LatLon_32719.shp", r"C:\Users\black\Documents\SINCA2\centros2.csv", radius_km=10)
df23 = build_df2(r"E:\Sick leave\microdatos_manzana\Centroide\Entidad_Promedios_32718.shp", r"E:\Sick leave\microdatos_manzana\Centroide\Coordenadas_LatLon_32718.shp", r"C:\Users\black\Documents\SINCA2\centros2.csv", radius_km=10)
df24 = build_df2(r"E:\Sick leave\microdatos_manzana\Centroide\Entidad_Promedios_32719.shp", r"E:\Sick leave\microdatos_manzana\Centroide\Coordenadas_LatLon_32719.shp", r"C:\Users\black\Documents\SINCA2\centros2.csv", radius_km=10)
# Append both results
df21["shp"] = 1
df22["shp"] = 2
df23["shp"] = 3
df24["shp"] = 4
final_df2 = pd.concat([df21, df22, df23, df24], ignore_index=True)

# Export to Excel
final_df2.to_excel(r"C:\Users\black\Documents\SINCA2\entidades_centros.xlsx", index=False)

OSError: Cannot save file into a non-existent directory: 'C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide'

In [7]:
final_df2.to_excel(r"C:\Users\black\Documents\SINCA2\entidades_centros.xlsx", index=False)

In [1]:
import os
import glob
import re
import rasterio
import geopandas as gpd
import pandas as pd
import numpy as np
from datetime import datetime

def process_rasters(shapefile, shp_label, raster_folders):
    gdf = gpd.read_file(shapefile).reset_index().rename(columns={"index": "Entity"})

    pattern = re.compile(r"(20\d{2}_[01]\d)")
    results = []

    for var, folder in raster_folders.items():
        raster_files = sorted(glob.glob(os.path.join(folder, "*.tif")))
        if not raster_files:
            continue

        # Open first raster to precompute row/col indices
        with rasterio.open(raster_files[0]) as src:
            rows_cols = []
            for x, y in zip(gdf.geometry.x, gdf.geometry.y):
                try:
                    row, col = src.index(x, y)
                    # Check bounds
                    if 0 <= row < src.height and 0 <= col < src.width:
                        rows_cols.append((row, col))
                    else:
                        rows_cols.append((None, None))
                except ValueError:
                    rows_cols.append((None, None))
            rowcol = np.array(rows_cols)

        for fpath in raster_files:
            match = pattern.search(os.path.basename(fpath))
            if not match:
                raise ValueError(f"Could not find YYYY_MM in filename: {fpath}")
            ym = match.group(1)
            year, month = map(int, ym.split("_"))

            with rasterio.open(fpath) as src:
                nodata = src.nodata

                for band in range(1, src.count + 1):
                    date = datetime(year, month, band)
                    arr = src.read(band)

                    values = []
                    for (r, c) in rowcol:
                        if r is None or c is None:
                            values.append(0)  # outside raster → 0
                        else:
                            val = arr[r, c]
                            if nodata is not None and val == nodata:
                                val = 0
                            elif np.isnan(val):
                                val = 0
                            values.append(val)

                    df_band = pd.DataFrame({
                        "Entity": gdf["Entity"],
                        "date": date,
                        "cod_comuna": gdf.get("CUT"),
                        "TOTAL_PERS": gdf.get("TOTAL_PERS"),
                        "value": values
                    })

                    results.append(df_band)

    return pd.concat(results, ignore_index=True)


In [ ]:
# Example usage
raster_folders = {
    "pr": r"C:\Users\black\Dropbox\tifs"
}

df = process_rasters(
    shapefile=r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\Entidad_Promedios_4326.shp",
    shp_label="Rural",
    raster_folders=raster_folders
)
df["value"]= (df["value"] + 30000)/100
df.to_csv(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\pr_rural.csv", index=False)

# Example usage
raster_folders = {
    "tmax": r"C:\Users\black\Dropbox\tifs2"
}

df = process_rasters(
    shapefile=r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\Entidad_Promedios_4326.shp",
    shp_label="Rural",
    raster_folders=raster_folders
)
df["value"]= df["value"]/600
df.to_csv(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\tmax_rural.csv", index=False)

# Example usage
raster_folders = {
    "tmin": r"C:\Users\black\Dropbox\tifs3"
}

df = process_rasters(
    shapefile=r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\Entidad_Promedios_4326.shp",
    shp_label="Rural",
    raster_folders=raster_folders
)
df["value"]= df["value"]/600
df.to_csv(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\tmin_rural.csv", index=False)


In [2]:

# Example usage
raster_folders = {
    "pr": r"C:\Users\black\Dropbox\tifs"
}

df = process_rasters(
    shapefile=r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\Centroides_Manzana_4326.shp",
    shp_label="Manzana",
    raster_folders=raster_folders
)
df["value"]= (df["value"] + 30000)/100
df.to_csv(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\pr_manzana.csv", index=False)

# Example usage
raster_folders = {
    "tmax": r"C:\Users\black\Dropbox\tifs2"
}

df = process_rasters(
    shapefile=r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\Centroides_Manzana_4326.shp",
    shp_label="Manzana",
    raster_folders=raster_folders
)
df["value"]= df["value"]/600
df.to_csv(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\tmax_manzana.csv", index=False)

# Example usage
raster_folders = {
    "tmin": r"C:\Users\black\Dropbox\tifs3"
}

df = process_rasters(
    shapefile=r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\Centroides_Manzana_4326.shp",
    shp_label="Manzana",
    raster_folders=raster_folders
)
df["value"]= df["value"]/600
df.to_csv(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\tmin_manzana.csv", index=False)
